In [1]:
import wandb
from sklearn.model_selection import train_test_split
import os,cv2
import numpy as np
import matplotlib.pyplot as plt
from torch.nn import *
import torch,torchvision
from tqdm import tqdm
device = 'cuda'
PROJECT_NAME = 'MNIST-Clf'

In [2]:
def load_data():
    labels = {}
    labels_r = {}
    idx = -1
    data = []
    for folder in tqdm(os.listdir('./data/')):
        idx += 1
        labels[folder] = idx
        labels_r[idx] = folder
        for file in os.listdir(f'./data/{folder}')[:2500]:
            img = cv2.imread(f'./data/{folder}/{file}')
            img = cv2.resize(img,(56,56))
            img = img / 255.0
            data.append([img,idx])
    np.random.shuffle(data)
    np.random.shuffle(data)
    X = []
    y = []
    for d in data:
        X.append(d[0])
        y.append(d[1])
    X_train,X_test,y_train,y_test = train_test_split(X,y,shuffle=False,test_size=0.125)
    return X_train,X_test,y_train,y_test,X,y,labels,labels_r,idx,data

In [3]:
X_train,X_test,y_train,y_test,X,y,labels,labels_r,idx,data = load_data()

100%|███████████████████████████████████████████| 10/10 [00:01<00:00,  6.96it/s]


In [4]:
X_train = torch.from_numpy(np.array(X_train)).view(-1,3,56,56).to(device).float()
X_test = torch.from_numpy(np.array(X_test)).view(-1,3,56,56).to(device).float()
y_train = torch.from_numpy(np.array(y_train)).to(device)
y_test = torch.from_numpy(np.array(y_test)).to(device)

In [5]:
def get_loss(model,X,y,criterion):
    preds = model(X)
    loss = criterion(preds,y)
    return loss.item()

In [6]:
def get_accuracy(model,X,y):
    correct = 0
    total = 0
    preds = model(X)
    for pred,y_batch in zip(preds,y):
        pred = int(torch.argmax(pred))
        if pred == y_batch:
            correct += 1
        total += 1
    acc = round(correct/total,3)*100
    print(correct,total)
    return acc

In [7]:
def get_pred(model):
    model.eval()
    for file in os.listdir('./test_data/'):
        img = cv2.imread(f'./test_data/{file}')
        img = cv2.resize(img,(56,56)) / 255.0
        img = torch.from_numpy(np.array(img)).to(device)
        pred = model(img.view(-1,3,56,56).float())
        plt.figure(figsize=(12,6))
        plt.imshow(img.cpu().detach().numpy())
        plt.title(f'{labels_r[int(torch.argmax(pred))]}')
        plt.savefig(f'./preds/{file}')
        plt.close()
    model.train()

In [16]:
class Model(Module):
    def __init__(self):
        super().__init__()
        self.max_pool2d = MaxPool2d((2,2),(2,2))
        self.activation = ReLU()
        self.conv1 = Conv2d(3,6,(5,5))
        self.conv1batchnorm = BatchNorm2d(6)
        self.conv2 = Conv2d(6,9,(5,5))
        self.conv2batchnorm = BatchNorm2d(9)
        self.conv3 = Conv2d(9,12,(5,5))
        self.conv3batchnorm = BatchNorm2d(12)
        self.conv4 = Conv2d(12,15,(2,2))
        self.conv4batchnorm = BatchNorm2d(15)
        self.linear1 = Linear(15*1*1,512)
        self.linear1batchnorm = BatchNorm1d(512)
        self.linear2 = Linear(512,1024)
        self.linear2batchnorm = BatchNorm1d(1024)
        self.linear3 = Linear(1024,2048)
        self.linear3batchnorm = BatchNorm1d(2048)
        self.linear4 = Linear(2048,1024)
        self.linear4batchnorm = BatchNorm1d(1024)
        self.output = Linear(1024,len(labels))
    
    def forward(self,X):
        preds = self.max_pool2d(self.activation(self.conv1batchnorm(self.conv1(X))))
        preds = self.max_pool2d(self.activation(self.conv2batchnorm(self.conv2(preds))))
        preds = self.max_pool2d(self.activation(self.conv3batchnorm(self.conv3(preds))))
        preds = self.max_pool2d(self.activation(self.conv4batchnorm(self.conv4(preds))))
        print(preds.shape)
        preds = preds.view(-1,15*1*1)
        preds =self.activation(self.linear1batchnorm(self.linear1(preds)))
        preds =self.activation(self.linear2batchnorm(self.linear2(preds)))
        preds =self.activation(self.linear3batchnorm(self.linear3(preds)))
        preds =self.activation(self.linear4batchnorm(self.linear4(preds)))
        preds = self.output(preds)
        return preds

In [17]:
model = Model().to(device)

RuntimeError: CUDA error: device-side assert triggered

In [ ]:
criterion = CrossEntropyLoss()

In [ ]:
from torch.optim import *

In [ ]:
optimizer = Adam(model.parameters(),lr=0.001)

In [13]:
epochs = 100

In [14]:
batch_size = 32

In [15]:
wandb.init(project=PROJECT_NAME,name='baseline')
wandb.watch(model)
for _ in tqdm(range(epochs)):
    torch.cuda.empty_cache()
    for idx in range(0,len(X_train),batch_size):
        torch.cuda.empty_cache()
        X_batch = X_train[idx:idx+batch_size].to(device).float().view(-1,3,56,56)
        y_batch = y_train[idx:idx+batch_size].to(device).long()
        preds = model(X_batch)
        loss = criterion(preds,y_batch)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    wandb.log({'Loss':get_loss(model,X_train,y_train,criterion)})
    wandb.log({'Val Loss':get_loss(model,X_test,y_test,criterion)})
    wandb.log({'Acc':get_accuracy(model,X_train,y_train)})
    wandb.log({'Val ACC':get_accuracy(model,X_test,y_test)})
    get_pred(model)
    for file in os.listdir('./preds/'):
        wandb.log({f'Img/{file}':wandb.Image(cv2.imread(f'./preds/{file}'))})
wandb.watch(model)
wandb.finish()

wandb: Currently logged in as: ranuga-d (use `wandb login --relogin` to force relogin)


  0%|                                                   | 0/100 [00:00<?, ?it/s]


torch.Size([32, 15, 1, 1])


/pytorch/aten/src/THCUNN/ClassNLLCriterion.cu:108: cunn_ClassNLLCriterion_updateOutput_kernel: block: [0,0,0], thread: [1,0,0] Assertion `t >= 0 && t < n_classes` failed.
/pytorch/aten/src/THCUNN/ClassNLLCriterion.cu:108: cunn_ClassNLLCriterion_updateOutput_kernel: block: [0,0,0], thread: [2,0,0] Assertion `t >= 0 && t < n_classes` failed.
/pytorch/aten/src/THCUNN/ClassNLLCriterion.cu:108: cunn_ClassNLLCriterion_updateOutput_kernel: block: [0,0,0], thread: [3,0,0] Assertion `t >= 0 && t < n_classes` failed.
/pytorch/aten/src/THCUNN/ClassNLLCriterion.cu:108: cunn_ClassNLLCriterion_updateOutput_kernel: block: [0,0,0], thread: [4,0,0] Assertion `t >= 0 && t < n_classes` failed.
/pytorch/aten/src/THCUNN/ClassNLLCriterion.cu:108: cunn_ClassNLLCriterion_updateOutput_kernel: block: [0,0,0], thread: [5,0,0] Assertion `t >= 0 && t < n_classes` failed.
/pytorch/aten/src/THCUNN/ClassNLLCriterion.cu:108: cunn_ClassNLLCriterion_updateOutput_kernel: block: [0,0,0], thread: [7,0,0] Assertion `t >= 0 

RuntimeError: CUDA error: device-side assert triggered